In [ ]:
import os
import numpy as np
import pylab as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
# Define data directories
train_dir = "Data_v2/train"
test_dir = "Data_v2/test"
validation_dir = "Data_v2/validation"

# Define image dimensions and batch size
img_height, img_width = 496, 369
batch_size = 8

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)


# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=["High_SNR", "Mid_SNR", "Low_SNR", "noise"],
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=["High_SNR", "Mid_SNR", "Low_SNR", "noise"],
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    classes=["High_SNR", "Mid_SNR", "Low_SNR", "noise"],
)

In [ ]:
# Define CNN model architecture
# Define CNN model architecture
model = Sequential(
    [
        Input(shape=(img_height, img_width, 3)),
        Conv2D(32, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(1024, activation="relu"),
        Dropout(0.5),
        Dense(512, activation="relu"),
        Dense(4, activation="softmax"),
    ]
)

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

# Get the accuracy values and epoch numbers from the history object
accuracy = history.history["accuracy"]
epochs = range(1, len(accuracy) + 1)

# Plot the accuracy vs epoch
plt.plot(epochs, accuracy, "b", label="Training Accuracy")
plt.title("Accuracy vs Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("\nTest accuracy:", test_acc)

In [ ]:
# Save the model
model.save("complex_spectrogram_model.h5")